In [2]:
from __future__ import absolute_import, division, print_function # Python2 compatibility
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

from The_Payne import utils
from The_Payne import training

import torch
from The_Payne import spectral_model
from The_Payne import fitting
import os

In [3]:
print(utils.__file__)

/home/zhangxu/anaconda3/lib/python3.7/site-packages/The_Payne/utils.py


In [4]:
wavelength = utils.load_wavelength_array(survey='galah')

In [5]:
wavelength.shape

(14304,)

In [6]:
training_labels, training_spectra, validation_labels, validation_spectra = \
utils.load_training_data(survey='galah',size=1000)

The model_grid has the following keywords:
dict_keys(['wavelength', 'teff', 'logg', 'feh', 'alpha_fe', 'c_fe', 'smod'])
Whole training set: 1000
Redefined training set (800) and validation set (200)
200


In [7]:
galah_model_grid = utils.get_model_grid()

The model_grid has the following keywords:
dict_keys(['wavelength', 'teff', 'logg', 'feh', 'alpha_fe', 'c_fe', 'smod'])


In [8]:
import scipy
interpolated_training_spectra = np.array([scipy.interp(wavelength, galah_model_grid['wavelength'], training_spectra[x]) for x in range(800)])
interpolated_validation_spectra = np.array([scipy.interp(wavelength, galah_model_grid['wavelength'], validation_spectra[x]) for x in range(200)])

In [9]:
training_labels = training_labels.astype(np.float64)
training_spectra = (interpolated_training_spectra).astype(np.float64)
validation_labels = validation_labels.astype(np.float64)
validation_spectra = (interpolated_validation_spectra).astype(np.float64)

In [10]:
training_spectra.shape

(800, 14304)

In [ ]:
#for vision 1.0 
#training_loss, validation_loss = training.neural_net(training_labels, training_spectra,\
#                                                     validation_labels, validation_spectra,\
#                                                     num_neurons = 300, num_steps=1e5, learning_rate=0.001)

In [11]:
training_loss, validation_loss = training.neural_net(training_labels, training_spectra, validation_labels, validation_spectra,\
             num_neurons = 300, num_steps=1e4, learning_rate=1e-4, batch_size=200, num_pixel=14304)

iter 0: training loss = 9518.314 validation loss = 9463.604
iter 100: training loss = 703.312 validation loss = 716.120
iter 200: training loss = 459.102 validation loss = 508.559
iter 300: training loss = 374.871 validation loss = 389.295
iter 400: training loss = 298.588 validation loss = 351.495
iter 500: training loss = 229.832 validation loss = 283.768
iter 600: training loss = 177.689 validation loss = 219.601
iter 700: training loss = 166.938 validation loss = 189.061
iter 800: training loss = 153.733 validation loss = 168.728
iter 900: training loss = 152.443 validation loss = 156.026
iter 1000: training loss = 137.963 validation loss = 146.899
iter 1100: training loss = 123.589 validation loss = 138.171
iter 1200: training loss = 117.212 validation loss = 130.132
iter 1300: training loss = 107.694 validation loss = 122.861
iter 1400: training loss = 94.144 validation loss = 117.275
iter 1500: training loss = 104.384 validation loss = 112.880
iter 1600: training loss = 85.602 v

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
tmp = np.load("training_loss.npz") 
training_loss = tmp["training_loss"]
validation_loss = tmp["validation_loss"]

plt.figure(figsize=(14, 4))
#plt.ylim(0,9)
plt.plot(np.arange(training_loss.size)*1000, training_loss, 'k', lw=0.5, label = 'Training set')
plt.plot(np.arange(training_loss.size)*1000, validation_loss, 'r', lw=0.5, label = 'Validation set')
plt.legend(loc = 'best', frameon = False, fontsize= 18)
plt.xlabel("Step", size=20)
plt.ylabel("Loss", size=20)

In [ ]:
tmp.files

###############################上面是训练模型,训练模型阶段没有使用mask

In [ ]:
##需要把训练好的模型放到程序里
NN_coeffs = utils.read_in_neural_network()
w_array_0, w_array_1, w_array_2, b_array_0, b_array_1, b_array_2, x_min, x_max = NN_coeffs

In [ ]:
spec_err = 1e-2*np.ones(len(wavelength))

In [ ]:
real_labels = scaled_labels = [5770, 4.44, 0.0,\
                               0.]

In [ ]:
scaled_labels[:] = (real_labels[:]-x_min)/(x_max-x_min) - 0.5
print(np.array(scaled_labels).shape)

In [ ]:
wavelength = torch.from_numpy(np.array(wavelength)).type(torch.FloatTensor)

In [ ]:
real_spec = spectral_model.get_spectrum_from_neural_net(scaled_labels = scaled_labels[:], NN_coeffs = NN_coeffs)

In [ ]:
#real_spec = utils.doppler_shift(wavelength, real_spec, scaled_labels[-1])   ##这个命令仅仅用来转成tensor格式

In [ ]:
plt.figure(figsize=(14, 4))
plt.plot(wavelength, real_spec, 'k', lw=0.5)
#plt.ylim(0.7, 1.05)

In [ ]:
data_spec = real_spec + 0.01*np.random.randn(len(real_spec))

In [ ]:
0.01*np.random.randn(len(real_spec))

In [ ]:
wavelength = utils.load_wavelength_array(survey='galah')

#mask = utils.galah_mask(wavelength, sme_like=False, cores_out=True)
#mask = utils.galah_mask(wavelength, sme_like=True, cores_out=True)

mask =np.ones(len(wavelength),dtype=bool)

#mask = utils.load_galah_mask()
#mask[np.abs(wavelength-4861.3230)<2.5]=False # do not use core of H_beta
#mask[np.abs(wavelength-6562.7970)<2.5]=False # do not use core of H_alpha

In [ ]:
mask

In [ ]:
popt, pcov, model_spec = fitting.fit_normalized_spectrum_single_star_model(\
                                norm_spec = data_spec, spec_err = spec_err,\
                                NN_coeffs = NN_coeffs, wavelength = wavelength, mask=mask, p0 = None)

plt.figure(figsize=(14, 4))
plt.plot(wavelength, data_spec, 'k', lw=0.5, label = '"data" spec')
plt.plot(wavelength, model_spec, 'r--', lw=0.5, label = 'best-fit model')
plt.xlim(6500, 6700)
plt.legend(loc = 'best', frameon = False, fontsize = 18)

In [ ]:
print(popt)

######################################################################上面是用假设参数作试验